In [5]:
from langchain.agents import tool

@tool
def get_weather(city: str) -> str:
  """Get the weather for a given city."""
  return f"It's always sunny in {city}!"

from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen3", temperature=0.3)

from langgraph.prebuilt import create_react_agent, ToolNode

agent_node = create_react_agent(llm, [get_weather])
tool_node = ToolNode([get_weather])


from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(MessagesState)
graph.add_node("agent", agent_node)
graph.add_node("tools", tool_node)

graph.add_edge(START, "agent")
def agent_router(state):
    # state["messages"] is a list of messages
    messages = state["messages"]
    print(f"Messages: {messages}")
    last = messages[-1]
    # Tool calls are usually FunctionMessage or have tool_call/tool_calls attribute
    if getattr(last, "tool_call", None) or getattr(last, "tool_calls", None):
        return "tools"
    # If it's an AIMessage and not a tool call, it's final
    return END

graph.add_conditional_edges("agent", path=agent_router)
graph.add_edge("tools", "agent")

app = graph.compile(checkpointer=MemorySaver())

from langchain_core.messages import HumanMessage

from langchain_core.runnables import RunnableConfig

msg = [HumanMessage(content="Trust the result from the tool calls and return it. It's test. What's the weather like in Kathmandu right now?")]
config: RunnableConfig = {"configurable": {"thread_id": "thread_1"}}

response = app.invoke({"messages": msg}, config=config)
print(response["messages"][-1].content)

Messages: [HumanMessage(content="Trust the result from the tool calls and return it. It's test. What's the weather like in Kathmandu right now?", additional_kwargs={}, response_metadata={}, id='7e688049-addd-48ca-8816-14deadbf5dc7'), AIMessage(content='<think>\nOkay, the user is asking about the weather in Kathmandu. I need to use the get_weather function. Let me check the function parameters. The required parameter is "city", so I should pass "Kathmandu" as the city argument. I\'ll structure the tool call accordingly.\n</think>\n\n', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-06-03T02:13:31.7351Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14276251292, 'load_duration': 4741672458, 'prompt_eval_count': 165, 'prompt_eval_duration': 6439091291, 'eval_count': 87, 'eval_duration': 3092415667, 'model_name': 'qwen3'}, id='run--575e04ea-31e0-43c4-b50a-e3050d9a3cd9-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Kathmandu'}, 'id': '1f4